# Aufgabe
Diese Daten stammen aus einer realen Produktionslinie in der Nähe von Detroit, Michigan. Ziel ist es, aus den verschiedenen Eingabedaten bestimmte Eigenschaften des Produktionsergebnisses der Linie vorherzusagen. Bei der Linie handelt es sich um einen kontinuierlichen Hochgeschwindigkeits-Fertigungsprozess mit parallelen und seriellen Stufen.

## Context

Die Daten stammen aus einem Produktionslauf, der sich über mehrere Stunden erstreckt. Liveline Technologies verfügt über eine große Menge dieser Art von Daten von mehreren Produktionslinien an verschiedenen Standorten.

## Herausforderung
Die Daten stammen aus einem mehrstufigen kontinuierlichen Fließfertigungsprozess. In der ersten Stufe arbeiten die Maschinen 1, 2 und 3 parallel und speisen ihre Ausgänge in einen Schritt ein, der die Ströme kombiniert. Die Outputs des Kombinators wird an 15 Stellen gemessen, die die Außenfläche des den Kombinator verlassenden Materials umgeben.

Primäres Ziel: Vorhersage der Messungen der Outputs der ersten Stufe.

## Danksagung

Das Liveline-Team möchte sich bei allen Technikern und Produktionsmitarbeitern bedanken, die bei dem Lauf und der Datenerfassung geholfen haben.

## Datenbeschreibung

Beschreibung des physischen Aufbaus:		
- Die Daten stammen aus einem kontinuierlichen Flussprozess.		
- Die Abtastrate beträgt 1 Hz.		
- In der ersten Stufe arbeiten die Maschinen 1, 2 und 3 parallel und speisen ihren Output in einen Schritt ein, der die Ströme kombiniert.		
- Der Output des Kombinators wird an 15 Stellen gemessen. Diese Messungen sind die primären Messungen, die vorhergesagt werden müssen.		
- Anschließend fließt die Outputs in eine zweite Stufe, in der die Maschinen 4 und 5 in Reihe arbeiten. Auch hier werden Messungen an denselben 15 Stellen vorgenommen. Dies sind die sekundären Messungen für die Vorhersage.	

Bisherige Erkenntnisse:
- Die Messungen sind verrauscht.		
- Jede Messung hat auch ein Ziel oder einen Sollwert (die Sollwerte sind in der ersten Datenzeile enthalten).		
- Ziel ist es, die Messungen (oder den Fehler gegenüber den Sollwerten) für so viele der 15 Messungen wie möglich vorherzusagen.
- Einige Messungen werden besser vorhersagbar sein als andere!		
- Die Vorhersage der Messungen nach der ersten Stufe ist von vorrangigem Interesse.		
- Die Vorhersage von Messungen nach der zweiten Stufe ist eine nette Sache, aber die Daten sind viel stärker verrauscht.		
		
Hinweis zu den Namenskonventionen für Variablen:	
- **~.C.Setpoint**:		Sollwert der Regelgröße
- **~.C.Actual**:		Istwert der Regelgröße
- **~.U.Actual**:		Istwert der ungeregelten Variablen
- **Others**:		Umwelt- oder Rohstoffvariablen, Zustände/Ereignisse, etc.
		
|Startspalte | Endspalte |	Beschreibung |
|------------|-----------|---------------|
|0|0|Zeitstempel|
|1|2|Umgebungsbedingungen der Fabrik|
|3|6|Erster Schritt, Maschine 1, Eigenschaften des Rohmaterials (Material, das in die Maschine 1 geht)|
|7|14|Erster Schritt, Maschine 1 Prozessvariablen|
|15|18|Zweiter Schritt, Maschine 2, Eigenschaften des Rohmaterials (Material, das in die Maschine 2 geht)|
|19|26|Erster Schritt, Maschine 2 Prozessvariablen|
|27|30|Zweiter Schritt, Maschine 3, Eigenschaften des Rohmaterials (Material, das in die Maschine 3 geht)|
|31|38|Erster Schritt, Maschine 3 Prozessvariablen|
|39|41|Prozessparameter der Kombinatorstufe. Hier werden die Outputs der Maschinen 1, 2 und 3 kombiniert.|
|42|71|PRIMÄRER OUTPUT ZUR STEUERUNG: Messungen von 15 Attributen (in mm), zusammen mit dem Sollwert oder Ziel für jedes Attribut|
|72|78|Zweiter Schritt, Maschine 4 Prozessvariablen|
|79|85|Zweiter Schritt, Maschine 5 Prozessvariablen|
|86|115|SEKUNDÄRER OUTPUT ZUR STEUERUNG: Messungen von 15 Attributen (in mm), zusammen mit dem Sollwert oder Ziel für jedes Attribut|
		
		
		
		


In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./data/continuous_factory_process.csv')

In [10]:
df.iloc[1000:1005]

,time_stamp,AmbientConditions.AmbientHumidity.U.Actual,AmbientConditions.AmbientTemperature.U.Actual,Machine1.RawMaterial.Property1,Machine1.RawMaterial.Property2,Machine1.RawMaterial.Property3,Machine1.RawMaterial.Property4,Machine1.RawMaterialFeederParameter.U.Actual,Machine1.Zone1Temperature.C.Actual,Machine1.Zone2Temperature.C.Actual,...,Stage2.Output.Measurement10.U.Actual,Stage2.Output.Measurement10.U.Setpoint,Stage2.Output.Measurement11.U.Actual,Stage2.Output.Measurement11.U.Setpoint,Stage2.Output.Measurement12.U.Actual,Stage2.Output.Measurement12.U.Setpoint,Stage2.Output.Measurement13.U.Actual,Stage2.Output.Measurement13.U.Setpoint,Stage2.Output.Measurement14.U.Actual,Stage2.Output.Measurement14.U.Setpoint
1000,2019-03-06 11:09:11,16.73,23.93,11.54,200,963.0,247,1249.33,72.1,71.8,...,7.92,7.93,5.66,5.65,2.06,1.85,3.39,2.89,8.19,11.71
1001,2019-03-06 11:09:12,16.73,23.93,11.54,200,963.0,247,1258.85,72.1,71.7,...,8.02,7.93,5.71,5.65,2.12,1.85,3.42,2.89,8.05,11.71
1002,2019-03-06 11:09:13,16.73,23.93,11.54,200,963.0,247,1273.09,72.1,71.7,...,7.83,7.93,5.82,5.65,2.10,1.85,3.41,2.89,0.00,11.71
1003,2019-03-06 11:09:14,16.73,23.93,11.54,200,963.0,247,1297.11,72.1,71.7,...,7.83,7.93,5.82,5.65,2.10,1.85,3.41,2.89,0.00,11.71
1004,2019-03-06 11:09:15,16.73,23.93,11.54,200,963.0,247,1274.94,72.1,71.7,...,7.88,7.93,5.76,5.65,2.11,1.85,3.40,2.89,10.97,11.71


Aus den Daten geht hervor, dass jede Zeile in der Tabelle steht ein Sample steht, die am Gerät entnommen wird, also ein Sample pro Sekunde. Jede Stichprobe umfasst das der Maschine zugeführte Material, den Sensorwert, der dem Zeitpunkt der Datenerfassung in der Maschine entspricht, und den Messwert des Produkts. Ziel ist es, die Messwerte aus den Material- und Maschinenwerten des Perzeptrons vorherzusagen. Hier gibt es jedoch ein Problem.

Dies ist auch eines der häufigsten Probleme bei der Qualitätskontrolle von Fertigungsprodukten, nämlich die Übereinstimmung zwischen Einflussfaktoren und Qualität.

Die Informationen, die zur gleichen Zeit an der Maschine gesammelt werden, sind nicht spezifisch für ein und dasselbe Produkt. Ein Produkt kann zu einem bestimmten Zeitpunkt des Produktionsprozesses nur an einer Stelle in der Linie erscheinen. Mit anderen Worten: Keiner der anderen Werte, die außerhalb des externen Geräts erfasst werden, ist eine Beschreibung des Produkts und hat im Prinzip nichts mit der Qualität des Produkts zu tun. Und das Fehlen einer Beschreibung der Pipelinegeschwindigkeit in der Dokumentation verhindert einen manuellen Abgleich.

Aufgrund des fehlenden Abgleichs kann man zwar allgemeine Qualitätsstatistiken für diesen Datensatz erstellen, aber es gibt keine Möglichkeit, die Produktqualität auf der Grundlage der Daten aus dem Produktionsprozess vorherzusagen.